In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import pickle
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pylab as plt

In [2]:
customer_df=pd.read_csv("/content/customers.csv")

In [3]:
customer_df.head(10)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
5,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
6,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,C
7,464347,Female,No,33,Yes,Healthcare,1.0,Low,3.0,Cat_6,D
8,465015,Female,Yes,61,Yes,Engineer,0.0,Low,3.0,Cat_7,D
9,465176,Female,Yes,55,Yes,Artist,1.0,Average,4.0,Cat_6,C


In [4]:
customer_df.columns

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

In [5]:
customer_df.dtypes

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object

In [6]:
customer_df.shape

(10695, 11)

In [7]:
customer_df.isnull().sum()

ID                    0
Gender                0
Ever_Married        190
Age                   0
Graduated           102
Profession          162
Work_Experience    1098
Spending_Score        0
Family_Size         448
Var_1               108
Segmentation          0
dtype: int64

In [8]:
customer_cleaned=customer_df.dropna(axis=0)

In [9]:
customer_cleaned.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [10]:
customer_cleaned.shape

(8819, 11)

In [11]:
customer_cleaned.drop(columns=['ID'], inplace=True)

<ipython-input-11-55645746f56c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_cleaned.drop(columns=['ID'], inplace=True)


In [12]:
customer_cleaned.drop(columns=['Var_1'], inplace=True)

<ipython-input-12-36cba2e3b45b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_cleaned.drop(columns=['Var_1'], inplace=True)


In [13]:
customer_cleaned.isnull().sum()

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Segmentation       0
dtype: int64

In [14]:
customer_cleaned['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [15]:
customer_cleaned['Ever_Married'].unique()

array(['No', 'Yes'], dtype=object)

In [16]:
customer_cleaned['Age'].unique()

array([22, 67, 56, 32, 33, 61, 55, 26, 19, 58, 41, 31, 79, 49, 18, 36, 35,
       45, 42, 83, 27, 28, 47, 40, 57, 76, 25, 48, 74, 59, 51, 30, 63, 52,
       39, 38, 60, 68, 86, 43, 80, 37, 46, 72, 69, 50, 29, 71, 78, 82, 70,
       23, 20, 21, 53, 75, 65, 89, 62, 66, 73, 77, 87, 84, 88, 85, 81])

In [17]:
customer_cleaned['Graduated'].unique()

array(['No', 'Yes'], dtype=object)

In [18]:
customer_cleaned['Profession'].unique()

array(['Healthcare', 'Engineer', 'Lawyer', 'Artist', 'Doctor',
       'Homemaker', 'Entertainment', 'Marketing', 'Executive'],
      dtype=object)

In [19]:
customer_cleaned['Spending_Score'].unique()

array(['Low', 'High', 'Average'], dtype=object)

In [20]:
customer_cleaned['Segmentation'].unique()

array(['D', 'B', 'C', 'A'], dtype=object)

In [21]:
categories_list=[['Male', 'Female'],
           ['No', 'Yes'],
           ['No', 'Yes'],
           ['Healthcare', 'Engineer', 'Lawyer', 'Artist', 'Doctor',
            'Homemaker', 'Entertainment', 'Marketing', 'Executive'],
           ['Low', 'Average', 'High']
           ]
enc = OrdinalEncoder(categories=categories_list)

In [22]:
customers_1 = customer_cleaned.copy()

In [23]:
customers_1[['Gender',
             'Ever_Married',
              'Graduated','Profession',
              'Spending_Score']] = enc.fit_transform(customers_1[['Gender',
                                                                 'Ever_Married',
                                                                 'Graduated','Profession',
                                                                 'Spending_Score']])

In [24]:
customers_1.dtypes

Gender             float64
Ever_Married       float64
Age                  int64
Graduated          float64
Profession         float64
Work_Experience    float64
Spending_Score     float64
Family_Size        float64
Segmentation        object
dtype: object

In [25]:
le = LabelEncoder()

In [26]:
customers_1['Segmentation'] = le.fit_transform(customers_1['Segmentation'])

In [27]:
customers_1.dtypes

Gender             float64
Ever_Married       float64
Age                  int64
Graduated          float64
Profession         float64
Work_Experience    float64
Spending_Score     float64
Family_Size        float64
Segmentation         int64
dtype: object

In [28]:
X=customers_1[['Gender','Ever_Married','Age','Graduated','Profession','Work_Experience','Spending_Score','Family_Size']].values


In [29]:
y = customers_1[['Segmentation']].values


In [30]:
ohc = OneHotEncoder()

In [31]:
ohc.fit(y)

OneHotEncoder()

In [32]:
y.shape

(8819, 1)

In [33]:
y1= ohc.transform(y).toarray()

In [34]:
y1.shape

(8819, 4)

In [35]:
y[0]
y1[0]

array([0., 0., 0., 1.])

In [36]:
X.shape

(8819, 8)

In [37]:
X_train,X_test,y_train,y_test=train_test_split(X,y1,test_size=0.33,random_state=50)



In [38]:
X_train.shape

(5908, 8)

In [39]:
scaler_age = MinMaxScaler()

In [40]:
scaler_age.fit(X_train[:,2].reshape(-1,1))

MinMaxScaler()

In [41]:
X_train_scaled = np.copy(X_train)
X_test_scaled = np.copy(X_test)

In [42]:
X_train_scaled[:,2] = scaler_age.transform(X_train[:,2].reshape(-1,1)).reshape(-1)
X_test_scaled[:,2] = scaler_age.transform(X_test[:,2].reshape(-1,1)).reshape(-1)

In [43]:
ai_brain = Sequential([
   Dense(4,input_shape=(8,)),
  Dense(8,activation='relu'),
  Dense(8,activation='relu'),
  Dense(4,activation='softmax'),
])

In [46]:
ai_brain.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])


In [47]:
ai_brain.fit(x=X_train_scaled,y=y_train,
             epochs=2000,batch_size=256,
             validation_data=(X_test_scaled,y_test),
             )


Epoch 1/2000
24/24 [==============================] - 3s 35ms/step - loss: 2.3121 - accuracy: 0.2395 - val_loss: 1.9644 - val_accuracy: 0.2357
Epoch 2/2000
24/24 [==============================] - 0s 9ms/step - loss: 1.7763 - accuracy: 0.2398 - val_loss: 1.6104 - val_accuracy: 0.2360
Epoch 3/2000
24/24 [==============================] - 0s 10ms/step - loss: 1.5191 - accuracy: 0.2493 - val_loss: 1.4467 - val_accuracy: 0.2810
Epoch 4/2000
24/24 [==============================] - 0s 9ms/step - loss: 1.4122 - accuracy: 0.3064 - val_loss: 1.3930 - val_accuracy: 0.3112
Epoch 5/2000
24/24 [==============================] - 0s 9ms/step - loss: 1.3794 - accuracy: 0.3170 - val_loss: 1.3754 - val_accuracy: 0.3167
Epoch 6/2000
24/24 [==============================] - 0s 10ms/step - loss: 1.3660 - accuracy: 0.3226 - val_loss: 1.3665 - val_accuracy: 0.3226
Epoch 7/2000
24/24 [==============================] - 0s 9ms/step - loss: 1.3580 - accuracy: 0.3306 - val_loss: 1.3605 - val_accuracy: 0.3318
Epo

In [67]:
metrics[['loss','val_loss']].plot()

KeyError: "None of [Index(['loss', 'val_loss'], dtype='object')] are in the [columns]"

In [51]:
x_test_predictions = np.argmax(ai_brain.predict(X_test_scaled), axis=1)


91/91 [==============================] - 1s 5ms/step


In [52]:
x_test_predictions.shape

(2911,)

In [53]:
y_test_truevalue = np.argmax(y_test,axis=1)

In [54]:
y_test_truevalue.shape

(2911,)

In [57]:
print(confusion_matrix(y_test_truevalue,x_test_predictions))

[[412 102 104 139]
 [254 146 197  89]
 [146 103 350  87]
 [241  54  45 442]]


In [59]:
metrics = pd.DataFrame(ai_brain.history.history)


In [61]:
metrics.head()

""


In [63]:
print(classification_report(y_test_truevalue,x_test_predictions))

              precision    recall  f1-score   support

           0       0.39      0.54      0.46       757
           1       0.36      0.21      0.27       686
           2       0.50      0.51      0.51       686
           3       0.58      0.57      0.57       782

    accuracy                           0.46      2911
   macro avg       0.46      0.46      0.45      2911
weighted avg       0.46      0.46      0.46      2911



In [64]:
x_single_prediction = np.argmax(ai_brain.predict(X_test_scaled[1:2,:]), axis=1)

1/1 [==============================] - 0s 38ms/step


In [65]:
print(x_single_prediction)


[2]


In [66]:
print(le.inverse_transform(x_single_prediction))

['C']
